In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import numpy as np
from itertools import product

In [101]:
T_list = torch.from_numpy(np.array([p for p in product(range(4), range(4), range(1))], dtype = np.float64))
k = torch.Tensor([1.,2.,0.]).to(T_list).reshape(1,-1)
H_T = torch.randn(T_list.shape[0], 10, 10)

In [105]:
T_list.shape

torch.Size([16, 3])

In [104]:
phase.shape

torch.Size([16, 1])

In [90]:
torch.einsum('Ta,ka->Tk', T_list, k)

RuntimeError: einsum(): the number of subscripts in the equation (2) does not match the number of dimensions (1) for operand 1 and no ellipsis was given

In [103]:
phase = torch.exp(2j * np.pi * torch.tensordot(T_list, k, dims = ([-1],[-1])))

In [92]:
phase.shape

torch.Size([16])

In [93]:
H_T.shape

torch.Size([16, 10, 10])

In [95]:
torch.tensordot(H_T.to(phase), phase, dims = ([0], [0])).shape

torch.Size([10, 10])

In [69]:
he=torch.einsum('Tab,Tk->kab', H_T.to(phase), phase)

In [82]:
td = torch.tensordot(H_T.to(phase), phase, dims = ([0], [0])).permute(2, 0, 1)

In [49]:
%%timeit 
torch.einsum('Ta,a->T', T_list, k)

10.2 µs ± 13.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [50]:
%%timeit 
torch.tensordot(T_list, k, dims = ([1],[0]))

5.9 µs ± 34.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [2]:
from ase.io import read
from ase.visualize import view
import numpy as np 
import torch 
import metatensor 
import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64)

In [3]:
from metatensor import Labels, TensorBlock, TensorMap
from mlelec.data.dataset import QMDataset
from mlelec.utils.twocenter_utils import fix_orbital_order

/home/pegolo/micromamba/envs/sci/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [4]:
orbitals = {'sto-3g': {6: [[1,0,0],[2,0,0],[2,1,1], [2,1,-1],[2,1,0]]}, 
            'def2svp': {6: [[1,0,0],[2,0,0],[3,0,0],[2,1,1], [2,1,-1],[2,1,0], [3,1,1], [3,1,-1],[3,1,0], [3,2,-2], [3,2,-1],[3,2,0], [3,2,1],[3,2,2]]},
           'gthszvmolopt': {6: [[2, 0, 0], 
                         [2, 1, 1], [2, 1, -1], [2, 1, 0]]
                            }
           }

In [80]:
workdir = '/home/pegolo/Software/my_mlelec/'
root = f'{workdir}/examples/data/periodic/c2'
ORBS = 'gthszvmolopt'
START = 0
STOP = 1
frames = read(f'{root}/C2_174.extxyz', slice(START, STOP))
for f in frames: 
    f.pbc = True

kmesh = [15,15,1]
kfock = [1/15*np.load(f"{root}/cp2k/fock_{i}.npy") for i in range(START, STOP)]
kover = [1/15*np.load(f"{root}/cp2k/overlap_{i}.npy") for i in range(START, STOP)]
for ifr in range(len(frames)):
    for ik, k in enumerate(kfock[ifr]):
        kfock[ifr][ik] = fix_orbital_order(k, frames[ifr], orbitals[ORBS]) #### TODO <<
        kover[ifr][ik] = fix_orbital_order(kover[ifr][ik], frames[ifr], orbitals[ORBS]) #### TODO <<

dataset = QMDataset(frames = frames, kmesh = kmesh, fock_kspace = kfock, overlap_kspace = kover, device = "cpu", orbs = orbitals[ORBS], orbs_name = ORBS)

WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



# Targets

In [81]:
from metatensor import load 
from mlelec.utils.twocenter_utils import _to_coupled_basis
from mlelec.utils.pbc_utils import matrix_to_blocks 
from mlelec.utils.symmetry import ClebschGordanReal

device = 'cpu'

def get_targets(dataset, device ="cpu", cutoff = None, target='fock', cg = None):
    if target.lower() == 'fock':
        matrices_negative = dataset._fock_realspace_negative_translations
    elif target.lower() == 'overlap':
        matrices_negative = dataset._overlap_realspace_negative_translations
    else: 
        raise ValueError('target must be fock or overlap')
    blocks = matrix_to_blocks(dataset, matrices_negative , device = 'cpu', cutoff = cutoff, all_pairs = True, target= target)
    coupled_blocks = _to_coupled_basis(blocks, skip_symmetry = True, device = device, translations = True, cg = cg)

    blocks = blocks.keys_to_samples('cell_shift_a')
    blocks = blocks.keys_to_samples('cell_shift_b')
    blocks = blocks.keys_to_samples('cell_shift_c')

    coupled_blocks = coupled_blocks.keys_to_samples('cell_shift_a')
    coupled_blocks = coupled_blocks.keys_to_samples('cell_shift_b')
    coupled_blocks = coupled_blocks.keys_to_samples('cell_shift_c')
    return blocks, coupled_blocks

In [94]:
cutoff = 6
CG = ClebschGordanReal(lmax = 3, device = device)
target_blocks, target_coupled_blocks = get_targets(dataset, cutoff = cutoff, cg = CG)

# Features

In [95]:
from mlelec.features.acdc import pair_features, single_center_features, twocenter_features_periodic_NH

hypers_pair = {'cutoff': cutoff,
               'max_radial': 10,
               'max_angular': 4,
               'atomic_gaussian_width': 0.3,
               'center_atom_weight': 1,
               "radial_basis": {"Gto": {}},
               "cutoff_function": {"ShiftedCosine": {"width": 0.1}}}

hypers_atom = {'cutoff': 4,
               'max_radial': 10,
               'max_angular': 4,
               'atomic_gaussian_width': 0.3,
               'center_atom_weight': 1,
               "radial_basis": {"Gto": {}},
               "cutoff_function": {"ShiftedCosine": {"width": 0.1}}}

return_rho0ij = False
both_centers = False
all_pairs = False
LCUT = 3

rhoij = pair_features(dataset.structures, hypers_atom, hypers_pair, order_nu = 1, all_pairs = all_pairs, both_centers = both_centers, mic = False,
                      kmesh = dataset.kmesh, device="cpu", lcut = LCUT, return_rho0ij = return_rho0ij, counter = dataset._translation_counter, 
                      T_dict = dataset._translation_dict)

if both_centers and not return_rho0ij:
    NU = 3
else:
    NU = 2
rhonui = single_center_features(dataset.structures, hypers_atom, order_nu = NU, lcut = LCUT, device = 'cpu',
                                feature_names = rhoij.property_names)

hfeat = twocenter_features_periodic_NH(single_center = rhonui, pair = rhoij, all_pairs = True)

{'cutoff': 6, 'max_radial': 10, 'max_angular': 4, 'atomic_gaussian_width': 0.3, 'center_atom_weight': 1, 'radial_basis': {'Gto': {}}, 'cutoff_function': {'ShiftedCosine': {'width': 0.1}}}
torch.Size([92, 1, 10]) 1
torch.Size([182, 1, 10]) 2
torch.Size([92, 3, 10]) 1
torch.Size([182, 3, 10]) 2
torch.Size([92, 5, 10]) 1
torch.Size([182, 5, 10]) 2
torch.Size([92, 7, 10]) 1
torch.Size([182, 7, 10]) 2
torch.Size([92, 9, 10]) 1
torch.Size([182, 9, 10]) 2


In [96]:
from metatensor import slice as mts_slice
trainidx = list(range(6))
testidx = list(range(6,10))
train_label = Labels(names=["structure"], values = np.array(trainidx).reshape(-1,1))
test_label = Labels(names=["structure"], values = np.array(testidx).reshape(-1,1))

hfeat_train = mts_slice(hfeat, 'samples', train_label)
hfeat_test =  mts_slice(hfeat, 'samples', test_label)
target_train = mts_slice(target_coupled_blocks, 'samples', train_label)
target_test = mts_slice(target_coupled_blocks, 'samples', test_label)


# Train

In [97]:
from mlelec.metrics import L2_loss, L2_kspace_loss

In [98]:
from mlelec.models.linear import LinearModelPeriodic
device='cpu'

In [99]:
model = LinearModelPeriodic(twocfeat = hfeat_train, 
                                  target_blocks = target_train,
                                  frames = dataset.structures, orbitals = dataset.basis, 
                                  device = device,
                           nhidden=16, nlayers=3)

model = model.double()

In [100]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5, patience = 30, verbose=True)

# optimizer = torch.optim.LBFGS(model.parameters(), lr=1e-4, history_size=10, 
#                     max_iter=4, 
#                     line_search_fn="strong_wolfe")

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, factor=0.5, patience=10, verbose=True
# )
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)# last_epoch=-1,

In [101]:
from mlelec.utils.pbc_utils import blocks_to_matrix
target_kspace = dataset.compute_matrices_kspace(blocks_to_matrix(target_train, dataset, cg = CG))

In [102]:
# losses =[]

def train_model(model, optimizer, loss_func = 'kspace', scheduler = None, nepoch = 1000, dataset = None):
    
    losses = []
    
    for epoch in range(nepoch):
        
        model.train(True)
        # def closure():
        #     optimizer.zero_grad()
        #     pred = model.forward()
        #     loss = L2_loss(pred, target_train)
        #     # loss = torch.linalg.norm(rmat - matrices)**2
        #     loss.backward()
        #     return loss
    
        # optimizer.step(closure)
        # train_loss = closure().item()
        #ADAM 
        optimizer.zero_grad()
        pred = model()
        # target.append(data['output'])
        # train_pred.append(pred)
        if loss_func == 'kspace':
            loss = L2_kspace_loss(pred, target_kspace, dataset, cg = CG)
        else:
            loss = L2_loss(pred, target_train)
        train_loss = loss.item()
        loss.backward()
        optimizer.step()
        
        # train_loss = loss_fn(torch.cat(train_pred), torch.cat(target))
        # print(train_loss - epoch_loss)
        losses.append(train_loss)
        if scheduler is not None:
            scheduler.step()
    
            # val_loss = loss_fn(torch.cat(val_pred), torch.cat(val))
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, train loss on all blocks {train_loss}")
        #     # validate 
        #     valpred = model.predict(hfeat_test, target_test)
        #     if loss_func == 'kspace':
        #         val_loss = L2_kspace_loss(valpred, target_test, dataset)
        #     else:
        #         val_loss = L2_loss(valpred, target_test)
            
        #     print(f"Epoch {epoch} val loss {val_loss}")

    return model

In [92]:
for g in optimizer.param_groups:
    g['lr'] = 1e-3

In [103]:
m = train_model(model, optimizer, nepoch = 20000, dataset = dataset)

Epoch 0, train loss on all blocks 8.821527092795904
Epoch 10, train loss on all blocks 6.468852516265518
Epoch 20, train loss on all blocks 6.194685044848282
Epoch 30, train loss on all blocks 5.795473398258038
Epoch 40, train loss on all blocks 5.175903720884815
Epoch 50, train loss on all blocks 4.698580369774456
Epoch 60, train loss on all blocks 4.32300506635068
Epoch 70, train loss on all blocks 4.045034636532593
Epoch 80, train loss on all blocks 3.7575880447607286
Epoch 90, train loss on all blocks 3.5833314900431104
Epoch 100, train loss on all blocks 3.555305064125962
Epoch 110, train loss on all blocks 3.4238795021693136
Epoch 120, train loss on all blocks 3.386747940197523
Epoch 130, train loss on all blocks 3.377197009452977
Epoch 140, train loss on all blocks 3.3004596818377956
Epoch 150, train loss on all blocks 3.305247898395405
Epoch 160, train loss on all blocks 3.2919865672194946
Epoch 170, train loss on all blocks 3.275875886539988
Epoch 180, train loss on all blocks

In [ ]:
%lprun -f train_model train_model(model, optimizer, nepoch = 1, dataset = dataset)

In [214]:
from mlelec.utils.twocenter_utils import _to_uncoupled_basis

In [231]:
from mlelec.utils.symmetry import ClebschGordanReal, _complex_clebsch_gordan_matrix, _real_clebsch_gordan_matrix

In [222]:
def comp_cg(lmax):
    return ClebschGordanReal(lmax, device = 'cpu')

In [228]:
%lprun -f _complex_clebsch_gordan_matrix _complex_clebsch_gordan_matrix(1, 1, 2, 'cpu')

Timer unit: 1e-09 s

Total time: 0.000335143 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/symmetry.py
Function: _complex_clebsch_gordan_matrix at line 228

Line #      Hits         Time  Per Hit   % Time  Line Contents
   228                                           def _complex_clebsch_gordan_matrix(l1: int, l2: int, L: int, device: str = None):
   229                                               """
   230                                               Computes the Clebsch-Gordan (CG) matrix for
   231                                               transforming complex-valued spherical harmonics.
   232                                               The CG matrix is computed as a 3D array of elements
   233                                               < l1 m1 l2 m2 | L M >
   234                                               where the first axis loops over m1, the second loops over m2,
   235                                               and the third one loops over M. Th

In [229]:
import wigners

In [230]:
%lprun -f wigners.clebsch_gordan_array wigners.clebsch_gordan_array(1, 1, 2)

Timer unit: 1e-09 s

Total time: 0.000228811 s
File: /home/pegolo/micromamba/envs/sci/lib/python3.11/site-packages/wigners/__init__.py
Function: clebsch_gordan_array at line 66

Line #      Hits         Time  Per Hit   % Time  Line Contents
    66                                           def clebsch_gordan_array(j1: int, j2: int, j3: int) -> np.ndarray:
    67                                               """
    68                                               Compute a full array of Clebsch-Gordan coefficient for the three given
    69                                               ``j``.
    70                                           
    71                                               The result is a 3-dimensional array with shape ``(2 * j1 + 1, 2 * j2 + 1, 2
    72                                               * j3 + 1)``.
    73                                               """
    74         1      10079.0  10079.0      4.4      array = np.zeros((2 * j1 + 1, 2 * j2 + 1, 2 * j

In [226]:
%lprun -f ClebschGordanReal.__init__ ClebschGordanReal(3, 'cpu')

Timer unit: 1e-09 s

Total time: 0.0424829 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/symmetry.py
Function: __init__ at line 288

Line #      Hits         Time  Per Hit   % Time  Line Contents
   288                                               def __init__(self, lmax: int, device: str = None):
   289         1        681.0    681.0      0.0          self.lmax = lmax
   290         1        271.0    271.0      0.0          self._cg = {}
   291         1        150.0    150.0      0.0          if device is not None:
   292         1        280.0    280.0      0.0              self.device = device
   293                                                   else:
   294                                                       self.device = "cuda" if torch.cuda.is_available() else "cpu"
   295                                           
   296         1        130.0    130.0      0.0          self.r2c = {}
   297         1        200.0    200.0      0.0          self.c2r = {}
   29

In [236]:
r2c = {}
c2r = {}
from mlelec.utils.symmetry import _real2complex
for L in range(0, 3 + 1):
    r2c[L] = _real2complex(L).to('cpu')
    c2r[L] = torch.conj(r2c[L]).T
rcg = _real_clebsch_gordan_matrix(1, 1, 2, r2c_l1=r2c[1], r2c_l2=r2c[1], c2r_L=c2r[2])

In [267]:
cg_M = {}
for M in range(-2, 3):
    cg_nonzero = torch.where(abs(rcg[:, :, M]) > 1e-15)
    cg_M[M] = torch.zeros(
        (len(cg_nonzero[0]), 3),
        # dtype=[(torch.int32, torch.int32, torch.int32)],
        device='cpu',
    )
    cg_M[M][:, 0] = cg_nonzero[0].type(torch.int)
    cg_M[M][:, 1] = cg_nonzero[1].type(torch.int)
    cg_M[M][:, 2] = rcg[cg_nonzero[0], cg_nonzero[1], M]

In [252]:
rcg.nonzero().shape

torch.Size([11, 3])

In [259]:
rcg[rcg.nonzero(as_tuple = True)]

tensor([-0.4082, -0.7071,  0.7071,  0.7071,  0.7071,  0.8165,  0.7071,  0.7071,
         0.7071, -0.4082,  0.7071])

In [269]:
cg_M[0]

tensor([[0.0000, 2.0000, 0.7071],
        [2.0000, 0.0000, 0.7071]])

In [ ]:
M=2
cg_nonzero = torch.where(abs(rcg[:, :, M]) > 1e-15)
cg_M = torch.zeros(
    (len(cg_nonzero[0]), 3),
    # dtype=[(torch.int32, torch.int32, torch.int32)],
    device='cpu',
)
cg_M[:, 0] = cg_nonzero[0].type(torch.int)
cg_M[:, 1] = cg_nonzero[1].type(torch.int)
cg_M[:, 2] = rcg[cg_nonzero[0], cg_nonzero[1], M]

In [271]:
from rascaline.utils import DensityCorrelations

In [273]:
dc = DensityCorrelations(3, 2)

In [276]:
target_blocks.keys

Labels(
    block_type  species_i  n_i  l_i  species_j  n_j  l_j
        0           6       1    0       6       1    0
        0           6       1    0       6       2    0
        0           6       1    0       6       2    1
        0           6       2    0       6       1    0
        0           6       2    0       6       2    0
        0           6       2    0       6       2    1
        0           6       2    1       6       1    0
        0           6       2    1       6       2    0
        0           6       2    1       6       2    1
        1           6       1    0       6       1    0
        -1          6       1    0       6       1    0
        1           6       1    0       6       2    0
        -1          6       1    0       6       2    0
        1           6       1    0       6       2    1
        -1          6       1    0       6       2    1
        1           6       2    0       6       1    0
        -1          6       2    0     

In [292]:
%lprun -f L2_kspace_loss L2_kspace_loss(pred, target_blocks, dataset) #, cg = CG)

Timer unit: 1e-09 s

Total time: 0.14945 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/metrics.py
Function: L2_kspace_loss at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def L2_kspace_loss(pred: Union[TensorMap], 
    11                                                              target: Union[TensorMap],
    12                                                              dataset: QMDataset,
    13                                                              cg: Optional[ClebschGordanReal] = None,
    14                                                              ):
    15                                               
    16         1       1032.0   1032.0      0.0      assert isinstance(target, TensorMap), "Target must be a TensorMap"
    17         1        181.0    181.0      0.0      assert isinstance(pred, TensorMap), "Prediction must be a TensorMap"
    18                                         

In [293]:
%lprun -f blocks_to_matrix blocks_to_matrix(pred, dataset, cg = CG)

Timer unit: 1e-09 s

Total time: 0.0903787 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/pbc_utils.py
Function: blocks_to_matrix at line 358

Line #      Hits         Time  Per Hit   % Time  Line Contents
   358                                           def blocks_to_matrix(blocks, dataset, device=None, return_negative=False, cg = None):
   359         1        601.0    601.0      0.0      if device is None:
   360         1       1092.0   1092.0      0.0          device = dataset.device
   361                                           
   362         1      81133.0  81133.0      0.1      if "L" in blocks.keys.names:
   363         1       2806.0   2806.0      0.0          from mlelec.utils.twocenter_utils import _to_uncoupled_basis
   364         1   19400914.0    2e+07     21.5          blocks = _to_uncoupled_basis(blocks, cg = cg)
   365                                           
   366         1       6222.0   6222.0      0.0      orbs_tot, orbs_offset = _orbs_offsets(da

In [288]:
%lprun -f blocks_to_matrix blocks_to_matrix(target_blocks, dataset) #, cg = CG)

Timer unit: 1e-09 s

Total time: 0.0522046 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/pbc_utils.py
Function: blocks_to_matrix at line 358

Line #      Hits         Time  Per Hit   % Time  Line Contents
   358                                           def blocks_to_matrix(blocks, dataset, device=None, return_negative=False, cg = None):
   359         1        491.0    491.0      0.0      if device is None:
   360         1       1122.0   1122.0      0.0          device = dataset.device
   361                                           
   362         1      78077.0  78077.0      0.1      if "L" in blocks.keys.names:
   363                                                   from mlelec.utils.twocenter_utils import _to_uncoupled_basis
   364                                                   blocks = _to_uncoupled_basis(blocks, cg = cg)
   365                                           
   366         1       5891.0   5891.0      0.0      orbs_tot, orbs_offset = _orbs_offsets(da

In [282]:
%lprun -f _to_uncoupled_basis _to_uncoupled_basis(target_coupled_blocks, cg = CG)

Timer unit: 1e-09 s

Total time: 0.0209131 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/twocenter_utils.py
Function: _to_uncoupled_basis at line 949

Line #      Hits         Time  Per Hit   % Time  Line Contents
   949                                           def _to_uncoupled_basis(
   950                                               blocks: TensorMap,
   951                                               # orbitals: Optional[dict] = None,
   952                                               cg: Optional[ClebschGordanReal] = None,
   953                                               device: str = "cpu",
   954                                               translations: bool = False,
   955                                           ):
   956                                           
   957         1        882.0    882.0      0.0      if cg is None:
   958                                                   lmax = max(blocks.keys["L"])
   959                               

In [212]:
%lprun -f blocks_to_matrix blocks_to_matrix(pred, dataset)

Timer unit: 1e-09 s

Total time: 0.115653 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/pbc_utils.py
Function: blocks_to_matrix at line 358

Line #      Hits         Time  Per Hit   % Time  Line Contents
   358                                           def blocks_to_matrix(blocks, dataset, device=None, return_negative=False):
   359         1        851.0    851.0      0.0      if device is None:
   360         1        661.0    661.0      0.0          device = dataset.device
   361                                           
   362         1      82966.0  82966.0      0.1      if "L" in blocks.keys.names:
   363         1       2265.0   2265.0      0.0          from mlelec.utils.twocenter_utils import _to_uncoupled_basis
   364         1   31888992.0    3e+07     27.6          blocks = _to_uncoupled_basis(blocks)
   365                                           
   366         1       7054.0   7054.0      0.0      orbs_tot, orbs_offset = _orbs_offsets(dataset.basis)
   367  

In [129]:
pred = model()

In [104]:
from mlelec.utils.pbc_utils import _atom_blocks_idx, _orbs_offsets
orbs_tot, orbs_offset = _orbs_offsets(dataset.basis)
atom_blocks_idx = _atom_blocks_idx(dataset.structures, orbs_tot)

In [63]:
from mlelec.utils.pbc_utils import blocks_to_matrix, move_cell_shifts_to_keys, OLD_blocks_to_matrix

In [13]:
from mlelec.utils.twocenter_utils import (
    _components_idx,
    ISQRT_2,
    _orbs_offsets,
    _atom_blocks_idx,
)

def moveit(blocks):
    if "cell_shift_a" not in blocks.keys.names:
        assert "cell_shift_b" not in blocks.keys.names, "Weird! keys contain 'cell_shift_b' but not 'cell_shift_a'."
        assert "cell_shift_c" not in blocks.keys.names, "Weird! keys contain 'cell_shift_c' but not 'cell_shift_a'."

        assert "cell_shift_a" in blocks.sample_names, "Cell shifts must be in samples."
        assert "cell_shift_b" in blocks.sample_names, "Cell shifts must be in samples."
        assert "cell_shift_c" in blocks.sample_names, "Cell shifts must be in samples."

        if "L" in blocks.keys.names:
            from mlelec.utils.twocenter_utils import _to_uncoupled_basis
            blocks = _to_uncoupled_basis(blocks)
        blocks = move_cell_shifts_to_keys(blocks) 
    return blocks

def opt_blocks_to_matrix(blocks, dataset, device=None, return_negative=False):
    if device is None:
        device = dataset.device

    orbs_tot, orbs_offset = _orbs_offsets(dataset.basis)
    atom_blocks_idx = _atom_blocks_idx(dataset.structures, orbs_tot)
    orbs_mult = {
        species: 
                {tuple(k): v
            for k, v in zip(
                *np.unique(
                    np.asarray(dataset.basis[species])[:, :2],
                    axis=0,
                    return_counts=True,
                )
            )
        }
        for species in dataset.basis
    }

    reconstructed_matrices_plus = []
    reconstructed_matrices_minus = []

    # Loop over frames
    for A, shifts in enumerate(dataset.realspace_translations):
        norbs = np.sum([orbs_tot[ai] for ai in dataset.structures[A].numbers])

        reconstructed_matrices_plus.append({T: torch.zeros(norbs, norbs, device = device) for T in shifts})
        reconstructed_matrices_minus.append({T: torch.zeros(norbs, norbs, device = device) for T in shifts})

    # loops over block types
    for key, block in blocks.items():
        block_type = key["block_type"]
        ai, ni, li = key["species_i"], key["n_i"], key["l_i"]
        aj, nj, lj = key["species_j"], key["n_j"], key["l_j"]
        
        # What's the multiplicity of the orbital type, ex. 2p_x, 2p_y, 2p_z makes the multiplicity 
        # of a p block = 3
        orbs_i = orbs_mult[ai]
        orbs_j = orbs_mult[aj]
        
        # The shape of the block corresponding to the orbital pair
        shapes = {
            (k1 + k2): (orbs_i[tuple(k1)], orbs_j[tuple(k2)])
            for k1 in orbs_i
            for k2 in orbs_j
        }
        # offset of the orbital (ni, li) within a block of atom i
        ioffset = orbs_offset[(ai, ni, li)] 
        # offset of the orbital (nj,lj) within a block of atom j
        joffset = orbs_offset[(aj, nj, lj)]

        i_end, j_end = shapes[(ni, li, nj, lj)]

        # loops over samples (structure, i, j)
        for sample, blockval in zip(block.samples, block.values):
            
            A = sample["structure"]
            i = sample["center"]
            j = sample["neighbor"]
            Tx, Ty, Tz = sample["cell_shift_a"], sample["cell_shift_b"], sample["cell_shift_c"]

            matrix_T_plus  = reconstructed_matrices_plus[A][Tx, Ty, Tz]
            matrix_T_minus = reconstructed_matrices_minus[A][Tx, Ty, Tz]
            i_start, j_start = atom_blocks_idx[(A, i, j)]
            values = blockval[:, :, 0].clone()

            if block_type == 0 or block_type == 2:

                matrix_T_plus[
                    i_start + ioffset : i_start + ioffset + i_end,
                    j_start + joffset : j_start + joffset + j_end,
                             ] = values

            if abs(block_type) == 1:
                values *= ISQRT_2
                if block_type == 1:
                    matrix_T_plus[
                        i_start + ioffset : i_start + ioffset + i_end,
                        j_start + joffset : j_start + joffset + j_end,
                    ] += values

                    matrix_T_minus[
                        j_start + ioffset : j_start + ioffset + i_end,
                        i_start + joffset : i_start + joffset + j_end,
                    ] += values
                    
                else:
                    
                    matrix_T_plus[
                        i_start + ioffset : i_start + ioffset + i_end,
                        j_start + joffset : j_start + joffset + j_end,
                    ] += values

                    matrix_T_minus[
                        j_start + ioffset : j_start + ioffset + i_end,
                        i_start + joffset : i_start + joffset + j_end,
                    ] -= values

    if return_negative:
        return reconstructed_matrices_plus, reconstructed_matrices_minus
    return reconstructed_matrices_plus

def opt_move_cell_shifts_to_keys(blocks):
    """ Move cell shifts when present in samples, to keys"""
    from metatensor import Labels, TensorBlock, TensorMap

    out_blocks = []
    out_block_keys = []

    sample_names = blocks.sample_names[:-3]
    
    for key, block in blocks.items():        
        translations = np.unique(block.samples.values[:, -3:], axis = 0)
        block_view = block.samples.view(["cell_shift_a", "cell_shift_b", "cell_shift_c"]).values
        for T in translations:
            # idx = np.where(np.all(np.isclose(np.array(block_view),np.array([T[0], T[1], T[2]])), axis = 1))[0]
            idx = np.where(np.all(np.isclose(np.array(block_view), T), axis = 1))[0]

            if len(idx):
                out_block_keys.append(list(key.values)+[T[0], T[1], T[2]])
                out_blocks.append(TensorBlock(
                        samples = Labels(
                            sample_names,
                            values = np.asarray(block.samples.values[idx])[:, :-3],
                        ),
                        values = block.values[idx],
                        components = block.components,
                        properties = block.properties,
                    ))
                
    return TensorMap(Labels(blocks.keys.names + ["cell_shift_a", "cell_shift_b", "cell_shift_c"], np.asarray(out_block_keys)), out_blocks)

In [10]:
%load_ext line_profiler

In [18]:
%lprun -f blocks_to_matrix blocks_to_matrix(target_blocks, dataset)

Timer unit: 1e-09 s

Total time: 0.483411 s
File: /home/pegolo/Software/my_mlelec/src/mlelec/utils/pbc_utils.py
Function: blocks_to_matrix at line 356

Line #      Hits         Time  Per Hit   % Time  Line Contents
   356                                           def blocks_to_matrix(blocks, dataset, device=None, return_negative=False):
   357         1        481.0    481.0      0.0      if device is None:
   358         1       1312.0   1312.0      0.0          device = dataset.device
   359                                                   
   360         1      74210.0  74210.0      0.0      if "cell_shift_a" not in blocks.keys.names:
   361         1      18736.0  18736.0      0.0          assert "cell_shift_b" not in blocks.keys.names, "Weird! keys contain 'cell_shift_b' but not 'cell_shift_a'."
   362         1      13816.0  13816.0      0.0          assert "cell_shift_c" not in blocks.keys.names, "Weird! keys contain 'cell_shift_c' but not 'cell_shift_a'."
   363               

In [19]:
%lprun -f opt_blocks_to_matrix opt_blocks_to_matrix(target_blocks, dataset)

Timer unit: 1e-09 s

Total time: 0.0761989 s
File: /tmp/ipykernel_3449353/866751707.py
Function: opt_blocks_to_matrix at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents

In [156]:
%lprun -f opt_move_cell_shifts_to_keys opt_move_cell_shifts_to_keys(target_blocks)

Timer unit: 1e-09 s

Total time: 0.212813 s
File: /tmp/ipykernel_3394842/1718792389.py
Function: opt_move_cell_shifts_to_keys at line 130

Line #      Hits         Time  Per Hit   % Time  Line Contents
   130                                           def opt_move_cell_shifts_to_keys(blocks):
   131                                               """ Move cell shifts when present in samples, to keys"""
   132         1      11372.0  11372.0      0.0      from metatensor import Labels, TensorBlock, TensorMap
   133                                           
   134         1        321.0    321.0      0.0      out_blocks = []
   135         1        200.0    200.0      0.0      out_block_keys = []
   136                                           
   137         1     236757.0 236757.0      0.1      sample_names = blocks.sample_names[:-3]
   138                                               
   139        28    1307199.0  46685.7      0.6      for key, block in blocks.items():        
   140

In [21]:
blocks_to_matrix(target_blocks, dataset) - blocks_to_matrix(target_blocks, dataset)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [66]:
blocks_to_matrix(pred, dataset)

[{(0,
   0,
   0): tensor([[-8.8181e+01, -2.3805e+01,  1.7539e-03,  5.1804e-03,  4.4494e-03,
           -2.1599e-03, -4.2054e+00,  6.9387e+00, -1.1433e-01,  1.2741e-01],
          [-2.3805e+01, -1.3864e+01,  3.1209e-02,  7.7723e-02,  1.0866e-01,
           -4.2054e+00, -6.2384e+00,  5.5694e+00, -9.7268e-02,  9.4786e-02],
          [ 1.7539e-03,  3.1209e-02, -4.7524e+00, -1.2551e-03, -1.3782e-01,
           -6.9387e+00, -5.5694e+00,  2.4265e+00, -7.4755e-02,  6.7299e-02],
          [ 5.1804e-03,  7.7723e-02, -1.2551e-03, -1.3138e+00, -2.2829e-03,
            1.1436e-01,  9.7270e-02, -7.4757e-02, -2.0365e+00, -1.1528e-03],
          [ 4.4494e-03,  1.0866e-01, -1.3782e-01, -2.2829e-03, -4.9473e+00,
           -1.2743e-01, -9.4767e-02,  6.7300e-02, -1.1528e-03, -2.1046e+00],
          [-2.1599e-03, -4.2054e+00, -6.9387e+00,  1.1433e-01, -1.2741e-01,
           -8.8181e+01, -2.3805e+01, -1.7539e-03, -5.1804e-03, -4.4494e-03],
          [-4.2054e+00, -6.2384e+00, -5.5694e+00,  9.7268e-02, -9

In [67]:
OLD_blocks_to_matrix(pred, dataset)

[{(0,
   0,
   0): tensor([[-8.8181e+01, -2.3805e+01,  1.7539e-03,  5.1804e-03,  4.4494e-03,
           -2.1599e-03, -4.2054e+00,  6.9387e+00, -1.1433e-01,  1.2741e-01],
          [-2.3805e+01, -1.3864e+01,  3.1209e-02,  7.7723e-02,  1.0866e-01,
           -4.2054e+00, -6.2384e+00,  5.5694e+00, -9.7268e-02,  9.4786e-02],
          [ 1.7539e-03,  3.1209e-02, -4.7524e+00, -1.2551e-03, -1.3782e-01,
           -6.9387e+00, -5.5694e+00,  2.4265e+00, -7.4755e-02,  6.7299e-02],
          [ 5.1804e-03,  7.7723e-02, -1.2551e-03, -1.3138e+00, -2.2829e-03,
            1.1436e-01,  9.7270e-02, -7.4757e-02, -2.0365e+00, -1.1528e-03],
          [ 4.4494e-03,  1.0866e-01, -1.3782e-01, -2.2829e-03, -4.9473e+00,
           -1.2743e-01, -9.4767e-02,  6.7300e-02, -1.1528e-03, -2.1046e+00],
          [-2.1599e-03, -4.2054e+00, -6.9387e+00,  1.1433e-01, -1.2741e-01,
           -8.8181e+01, -2.3805e+01, -1.7539e-03, -5.1804e-03, -4.4494e-03],
          [-4.2054e+00, -6.2384e+00, -5.5694e+00,  9.7268e-02, -9